In [2]:
import pandas as pd


# BASE DATABRICKS

### ARQUIVO ==> otmlog_Fato_VolumeFrotaPropria.csv

In [141]:
arquivo_OTMLOG_DATABRICKS = 'C:\\Temp\\vol_frota_sergio\\vol_frota_v2\\otmlog_Fato_VolumeFrotaPropria.csv'

dfOTMLOG_DATABRICKS = pd.read_csv(arquivo_OTMLOG_DATABRICKS
                                  , sep=','
                                  ,usecols=[0,1,2,3,4,6]
                                 ,names=[
                                'CodRelatorio',
                                'CodItem',
                                'CodDocVenda',
                                'NumItemDocVenda',
                                'Qtd',
                                 'QtdPesoLiqKG']
                                  ,dtype={'CodRelatorio':'str',
                                'CodItem':'str',
                                'CodDocVenda':'str',
                                'NumItemDocVenda':'str',
                                'Qtd':'float',
                                'QtdPesoLiqKG':'float'}
                                  ,header=0
                                 )


dfOTMLOG_DATABRICKS['Chave_DATABRICKS'] = dfOTMLOG_DATABRICKS['CodRelatorio'] + dfOTMLOG_DATABRICKS['CodItem'].astype(int).astype(str)

In [156]:

#dfOTMLOG_DATABRICKS.info()
dfOTMLOG_DATABRICKS[dfOTMLOG_DATABRICKS['CodDocVenda'] == '0032100938']

,CodRelatorio,CodItem,CodDocVenda,NumItemDocVenda,Qtd,QtdPesoLiqKG,Chave_DATABRICKS
194393,1009389206,000030,0032100938,000020,5.0,225.0,100938920630
735233,1009389206,000010,0032100938,000010,309.0,4017.0,100938920610


# BASE ALEX PAIVA
### ARQUIVO ==> ZTSD088_06_22_OtimLog_ALEX.xlsx

In [200]:
arquivo_OTMLOG_ALEX = 'C:\\Temp\\vol_frota_sergio\\vol_frota_v2\\ZTSD088_06_22_OtimLog_ALEX.xlsx'

dfOTMLOG_ALEX = pd.read_excel(arquivo_OTMLOG_ALEX
                             ,sheet_name='Filtrado'
                              ,usecols=[0,1,2,10,12,14,17,18]
                                 ,names=['CodRelatorio',
                                'CodItem',
                                'Chave',
                                'CodMaterial',
                                'Qtd',
                                'VolCIF',
                                'QtdPesoLiqKG',
                                'CodDocVenda']
                                ,dtype={'CodRelatorio':'str',
                                'CodItem':'str',
                                'Chave':'str',
                                'CodMaterial':'str',
                                'Qtd':'float',
                                'VolCIF':'float',
                                'QtdPesoLiqKG':'float',
                                'CodDocVenda':'str'}
                                 )

dfOTMLOG_ALEX

,CodRelatorio,CodItem,Chave,CodMaterial,Qtd,VolCIF,QtdPesoLiqKG,CodDocVenda
0,1008776067,20,100877606720,90034,135.0,0.0,1.755,0029731929
1,1009379307,10,100937930710,96809,130.0,130.0,0.130,0031965046
2,1009379307,20,100937930720,96809,260.0,260.0,0.260,0031885328
3,1009379307,30,100937930730,96809,180.0,180.0,0.180,0031885290
4,1009379307,40,100937930740,96809,136.0,136.0,0.136,0031973558
...,...,...,...,...,...,...,...,...
150021,1009535755,20,100953575520,90034,250.0,3250.0,3.250,0032544060
150022,1009535757,20,100953575720,90034,250.0,3250.0,3.250,0032544061
150023,1009535760,20,100953576020,90035,10.0,200.0,0.200,0032492111
150024,1009535774,10,100953577410,96809,200.0,200.0,0.200,0032560361


# REMOVE "." DA COLUNA ==> QtdPesoLiqKG

In [161]:
dfOTMLOG_ALEX['QtdPesoLiqKG'] = dfOTMLOG_ALEX['QtdPesoLiqKG'].astype(str).apply(lambda x: x.replace('.','')).astype(float)

# CONVERT QTD DE ACORDO COM O MATERIAL

In [260]:
dfOTMLOG_ALEX.loc[(dfOTMLOG_ALEX['CodMaterial'] == '90034', 'QtdCalculada')] = round((dfOTMLOG_ALEX['Qtd'] * 13),2) # * 1000
                  
dfOTMLOG_ALEX.loc[(dfOTMLOG_ALEX['CodMaterial'] == '96809', 'QtdCalculada')] = round((dfOTMLOG_ALEX['Qtd']  ),2) #* 1000
                  
dfOTMLOG_ALEX.loc[(dfOTMLOG_ALEX['CodMaterial'] == '90036', 'QtdCalculada')] = round((dfOTMLOG_ALEX['Qtd'] * 45),2) #* 1000
                  
dfOTMLOG_ALEX.loc[(dfOTMLOG_ALEX['CodMaterial'] == '90035', 'QtdCalculada')] = round((dfOTMLOG_ALEX['Qtd'] * 20),2) #* 1000
                  
dfOTMLOG_ALEX.loc[(dfOTMLOG_ALEX['CodMaterial'] == '90033', 'QtdCalculada')] = round((dfOTMLOG_ALEX['Qtd'] * 5),2) #* 1000
                  
dfOTMLOG_ALEX.loc[(dfOTMLOG_ALEX['CodMaterial'] == '90031', 'QtdCalculada')] = round((dfOTMLOG_ALEX['Qtd'] * 2)) # * 1000

# COMPARAÇÃO ARQUIVO ALEX VS ARQUIVO DATABRICKS

In [261]:
from decimal import Decimal

result01 = pd.merge(left=dfOTMLOG_ALEX
                    , right=dfOTMLOG_DATABRICKS
                    , left_on='Chave'
                    , right_on='Chave_DATABRICKS'
                    ,how='left'
                    )#[['CodRelatorio_x','CodItem_x','Chave','Analise']]

result01.loc[(result01['Chave'] == result01['Chave_DATABRICKS']) &
             (result01['QtdCalculada'] == result01['QtdPesoLiqKG_y']),'Comparacao'] = 'Registro no Arquivo e VolCIF Igual'

result01.loc[(result01['Chave'] == result01['Chave_DATABRICKS']) &
             (result01['QtdCalculada'] != result01['QtdPesoLiqKG_y']),'Comparacao'] = 'Registro no Arquivo e VolCIF Diferent'

result01.loc[result01['Chave'] != result01['Chave_DATABRICKS'] ,'Comparacao'] = 'Não Encontrado no Arquivo'

result01.groupby(['Comparacao']).agg(Qtd=('Comparacao','count'))

,Qtd
Comparacao,
Não Encontrado no Arquivo,851
Registro no Arquivo e VolCIF Diferent,1
Registro no Arquivo e VolCIF Igual,149174


In [259]:
result01.groupby(['CodMaterial']).agg(Qtd=('CodMaterial','count'))

,Qtd
CodMaterial,
90031,7
90033,424
90034,46553
90035,20171
90036,18548
96809,64323


In [262]:

result01[(result01['Comparacao'] == 'Registro no Arquivo e VolCIF Diferent')][['Chave'
                                                                               ,'CodRelatorio_x'
                                                                               ,'CodItem_x'
                                                                               ,'CodDocVenda_x'
                                                                               ,'NumItemDocVenda'
                                                                               ,'CodMaterial'
                                                                               ,'Qtd_x'
                                                                               ,'Qtd_y'
                                                                               ,'QtdCalculada'
                                                                               ,'QtdPesoLiqKG_y']]


,Chave,CodRelatorio_x,CodItem_x,CodDocVenda_x,NumItemDocVenda,CodMaterial,Qtd_x,Qtd_y,QtdCalculada,QtdPesoLiqKG_y
183,100939897650,1009398976,50,0032063588,000050,90036,30.857,30.857,1388.56,1388.57


# BASE ODS DADOS DE ORDEM DE VENDA 2022

In [119]:
arquivo_BASE_ODS = 'C:\\Temp\\vol_frota_sergio\\vol_frota_v2\\base_ods.csv'

dfBASE_ODS = pd.read_csv(arquivo_BASE_ODS
                                  , sep=';'
                                 ,names=['AnoMes'
                                , 'CodRelatorio'
                                , 'CodItem'
                                , 'Qtd'
                                , 'CodDocVenda'
                                , 'NumItemDocVenda'
                                , 'Id_MotivoRecusa'
                                , 'CodDFEstornado'
                                , 'Analise']
                         ,dtype={'AnoMes':'str'
                                , 'CodRelatorio':'str'
                                , 'CodItem':'str'
                                , 'Qtd':'str'
                                , 'CodDocVenda':'str'
                                , 'NumItemDocVenda':'str'
                                , 'Id_MotivoRecusa':'str'
                                , 'CodDFEstornado':'str'
                                , 'Analise':'str'}
                                  ,header=0
                                 )

dfBASE_ODS['Chave_ODS'] = dfBASE_ODS['CodRelatorio'] + dfBASE_ODS['CodItem'].astype(int).astype(str)

In [124]:
dfOTMLOG_ALEX_NE = result01[result01['Comparacao'] == 'Não Encontrado no Arquivo']

dfOTMLOG_ALEX_NE

,CodRelatorio_x,CodItem_x,Chave,VolCIF,CodDocVenda_x,CodRelatorio_y,CodItem_y,CodDocVenda_y,NumItemDocVenda,qtd,Chave_DATABRICKS,Comparacao
0,1008776067,20,100877606720,0.0,0029731929,NaN,NaN,NaN,NaN,NaN,NaN,Não Encontrado no Arquivo
42,1009390607,50,100939060750,0.0,0031732644,NaN,NaN,NaN,NaN,NaN,NaN,Não Encontrado no Arquivo
51,1009390607,140,1009390607140,0.0,0032069013,NaN,NaN,NaN,NaN,NaN,NaN,Não Encontrado no Arquivo
161,1009398072,20,100939807220,0.0,0032047610,NaN,NaN,NaN,NaN,NaN,NaN,Não Encontrado no Arquivo
828,1009402255,430,1009402255430,0.0,0032078822,NaN,NaN,NaN,NaN,NaN,NaN,Não Encontrado no Arquivo
...,...,...,...,...,...,...,...,...,...,...,...,...
149714,1009533842,20,100953384220,0.0,0032529216,NaN,NaN,NaN,NaN,NaN,NaN,Não Encontrado no Arquivo
149715,1009533842,30,100953384230,0.0,0032529219,NaN,NaN,NaN,NaN,NaN,NaN,Não Encontrado no Arquivo
149729,1009533925,10,100953392510,0.0,0032504843,NaN,NaN,NaN,NaN,NaN,NaN,Não Encontrado no Arquivo
149870,1009534488,10,100953448810,0.0,0032563547,NaN,NaN,NaN,NaN,NaN,NaN,Não Encontrado no Arquivo


# VALIDANDO OS 851 NÃO ENCONTRADOS NO ARQUIVO...

In [132]:

result02 = pd.merge(left=dfOTMLOG_ALEX_NE
                    , right=dfBASE_ODS
                    , left_on='Chave'
                    , right_on='Chave_ODS'
                    ,how='left'
                    )[['Chave'
                       , 'CodRelatorio_x'
                       , 'CodItem_x'
                       , 'CodDocVenda_x'
                       , 'NumItemDocVenda_y'
                       , 'Id_MotivoRecusa'
                       , 'CodDFEstornado'
                       , 'Analise']]

result02['Analise'].fillna('NE',inplace=True)

result02.drop_duplicates(subset = ['Chave']
                         , keep = 'last').reset_index(drop = True) \
                         .groupby(['Analise']) \
                         .agg(Qtd=('Analise','count'))

,Qtd
Analise,
Fatura Estornada,67
NE,30
OV Com Motivo de Recusa,754
